In [144]:
import pandas as pd

In [145]:
# Loading Data and Dropping Null Values

# data = pd.read_csv('Data/Jan/202401-divvy-tripdata.csv')
# data = pd.read_csv('Data/Feb/202402-divvy-tripdata.csv')
# data = pd.read_csv('Data/March/202403-divvy-tripdata.csv')
# data = pd.read_csv('Data/April/202404-divvy-tripdata.csv')
# data = pd.read_csv('Data/May/202405-divvy-tripdata.csv')
# data = pd.read_csv('Data/June/202406-divvy-tripdata.csv') 
# data = pd.read_csv('Data/July/202407-divvy-tripdata.csv') 
# data = pd.read_csv('Data/Aug/202408-divvy-tripdata.csv')
# data = pd.read_csv('Data/Sept/202409-divvy-tripdata.csv')
# data = pd.read_csv('Data/Oct/202410-divvy-tripdata.csv')
# data = pd.read_csv('Data/Nov/202411-divvy-tripdata.csv')
data = pd.read_csv('Data/Dec/202412-divvy-tripdata.csv')

data.dropna(inplace=True)   
data.reset_index(drop=True, inplace=True)
# print(data.info())


print((data['start_station_name' ].value_counts(normalize=True)*100).head(10))

start_station_name
Kingsbury St & Kinzie St        1.283562
Clinton St & Washington Blvd    1.151228
Clinton St & Madison St         1.034193
Canal St & Madison St           1.001300
Clark St & Elm St               0.894209
Clinton St & Jackson Blvd       0.859787
Wells St & Elm St               0.788648
Daley Center Plaza              0.787883
State St & Chicago Ave          0.778704
Larrabee St & Kingsbury St      0.771820
Name: proportion, dtype: float64


In [146]:
# Taking a random sample of 10% of the data

sample = data.sample(frac=0.1, random_state=42)
sample.reset_index(drop=True, inplace=True) 
# print(sample.info())

print((sample['start_station_name' ].value_counts(normalize=True)*100).head(10))

start_station_name
Kingsbury St & Kinzie St        1.239195
Clinton St & Washington Blvd    1.093858
Canal St & Madison St           1.040312
Clinton St & Madison St         1.025013
Clark St & Elm St               0.879676
Clinton St & Jackson Blvd       0.864377
Daley Center Plaza              0.833780
Wells St & Elm St               0.803182
Larrabee St & Kingsbury St      0.780234
State St & Chicago Ave          0.749637
Name: proportion, dtype: float64


In [147]:
# Encode 'rideable_type' and 'member_casual' as categorical codes

sample['rideable_type_code'] = sample['rideable_type'].astype('category').cat.codes
sample['member_casual_code'] = sample['member_casual'].astype('category').cat.codes

In [148]:
print(sample.columns)

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'rideable_type_code', 'member_casual_code'],
      dtype='object')


In [149]:
# Convert started_at and ended_at to datetime
sample['started_at'] = pd.to_datetime(sample['started_at'], errors='coerce')
sample['ended_at']   = pd.to_datetime(sample['ended_at'], errors='coerce')

# Ensure latitude and longitude columns are float
lat_lng_cols = ['start_lat', 'start_lng', 'end_lat', 'end_lng']
sample[lat_lng_cols] = sample[lat_lng_cols].astype(float)

# Ensure station ID columns are string
station_id_cols = ['start_station_id', 'end_station_id']
sample[station_id_cols] = sample[station_id_cols].astype(str)

In [150]:
sample['ride_length'] = (sample['ended_at'] - sample['started_at']).dt.total_seconds() / 60 
sample['hour_of_day'] = sample['started_at'].dt.strftime('%I %p')
sample['day_of_week'] = sample['started_at'].dt.day_name()

In [151]:
# Remove rides with negative or excessively long ride lengths
sample = sample[(sample['ride_length'] >= 0) & 
                                (sample['ride_length'] <= 1440)]

In [152]:
# Filter to Chicago area based on lat/lng bounds
sample = sample[(sample['start_lat'].between(41.6, 42.1)) &
                                (sample['start_lng'].between(-88.0, -87.5)) &
                                (sample['end_lat'].between(41.6, 42.1)) &
                                (sample['end_lng'].between(-88.0, -87.5))]  


In [153]:
# Encode 'rideable_type' and 'member_casual' as categorical codes
reordered_columns = ['ride_id',
 'rideable_type', 'rideable_type_code',
 'started_at', 'ended_at',
 'ride_length', 'day_of_week', 'hour_of_day',
 'start_station_name', 'start_station_id', 'start_lat', 'start_lng',
 'end_station_name', 'end_station_id', 'end_lat', 'end_lng',
 'member_casual', 'member_casual_code']

sample = sample[reordered_columns]

print(sample.head())

            ride_id  rideable_type  rideable_type_code  \
0  4A7F69FDA13729A2  electric_bike                   1   
1  00B7DC79929A333C  electric_bike                   1   
2  6F1D24E0F6BE31DD  electric_bike                   1   
3  70A49333FDCFF51F  electric_bike                   1   
4  7FB4E0425523CDAC  electric_bike                   1   

               started_at                ended_at  ride_length day_of_week  \
0 2024-12-17 10:02:11.584 2024-12-17 10:06:00.595     3.816850     Tuesday   
1 2024-12-13 22:39:20.815 2024-12-13 22:51:08.242    11.790450      Friday   
2 2024-12-02 14:40:42.996 2024-12-02 14:50:48.107    10.085183      Monday   
3 2024-12-02 05:58:01.727 2024-12-02 06:15:01.504    16.996283      Monday   
4 2024-12-21 01:46:34.476 2024-12-21 01:58:01.605    11.452150    Saturday   

  hour_of_day              start_station_name start_station_id  start_lat  \
0       10 AM  Southport Ave & Wrightwood Ave     TA1307000113  41.928773   
1       10 PM      Californi

In [ ]:
# Converting the cleaned sample to a CSV file

# sample.to_csv('Data/June/Divvy_cleaned_June_2024.csv')
# sample.to_csv('Data/July/Divvy_cleaned_July_2024.csv')
# sample.to_csv('Data/Aug/Divvy_cleaned_Aug_2024.csv')
# sample.to_csv('Data/Sept/Divvy_cleaned_Sept_2024.csv')
# sample.to_csv('Data/Nov/Divvy_cleaned_Nov_2024.csv')
# sample.to_csv('Data/Dec/Divvy_cleaned_Dec_2024.csv')